In [ ]:
import pandas as pd
import re, time, requests
from selenium import webdriver
from bs4 import BeautifulSoup
import json
import os 


path='./104_comapny'
if not os.path.exists(path):
    os.mkdir(path)


    
headers = {'User-Agent': 'your user-agent'}

my_params = {'ro':'1', # 限定全職的工作，如果不限定則輸入0
             'keyword':'數據分析', # 想要查詢的關鍵字
             'area':'6001001000', # 限定在台北的工作
             'isnew':'30', # 只要最近一個月有更新的過的職缺
             'mode':'l'} # 清單的瀏覽模式

url = requests.get('https://www.104.com.tw/jobs/search/?' , my_params, headers = headers).url
driver = webdriver.Chrome()
driver.get(url)

# 網頁的設計方式是滑動到下方時，會自動加載新資料，在這裡透過程式送出Java語法幫我們執行「滑到下方」的動作
for i in range(20):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(10)


soup = BeautifulSoup(driver.page_source, 'html.parser')
jobs = soup.findAll('article',class_='js-job-item')
JobList = pd.DataFrame()

for job in jobs: 

    job_url='http:'+job.find('a').get('href')
    referer=job_url.split('?')[0]
    job_url_json='https://www.104.com.tw/job/ajax/content/'+referer.split('/')[-1]
    headers2 = {'User-Agent': 'Mozilla/5.0': referer}
    job_res=requests.get(job_url_json, headers=headers2)
    jsonData=json.loads(job_res.text)
    skills=jsonData['data']['condition']['specialty']

    skill_list=[]
    for i in range(len(skills)):
        if len(skills)!=0:
            skill_item=skills[i]['description']
            skill_list.append(skill_item)


    df = pd.DataFrame(
        data = [{
            'Company_Name':job.get('data-cust-name'),
            'Job_Opening':job.find('a',class_="js-job-link").text,
            'Industry':jsonData['data']['industry'],
            'Salary':jsonData['data']['jobDetail']['salary'],
            'Job_Description':jsonData['data']['jobDetail']['jobDescription'],
            'Other_Requirement':jsonData['data']['condition']['other'],
            'Job_Url':'http:'+job.find('a').get('href'),
            'Skill_list':skill_list}],
          columns = ['Company_Name','Job_Opening','Industry','Salary','Job_Description','Other_Requirement','Job_Url','Skill_list'])

    JobList = JobList.append(df, ignore_index=True)

JobList_Skill=JobList.Skill_list.apply(pd.Series) \
    .merge(JobList, left_index = True, right_index = True) \
    .drop(["Skill_list"], axis = 1) \
    .melt(id_vars = ['Company_Name', 'Job_Opening','Industry','Salary','Job_Description','Other_Requirement','Job_Url'], value_name = "Skill_list") \
    .drop("variable", axis = 1) \
    .fillna(value='Not Required') 

JobList2=pd.concat([JobList_Skill, pd.get_dummies(JobList_Skill['Skill_list'])], 1) \
    .groupby(['Company_Name', 'Job_Opening','Industry','Salary','Job_Description','Other_Requirement','Job_Url']).sum().reset_index() \
    .drop(["Not Required"], axis = 1)

JobList2.to_csv('./104_comapny/jobresearch2.csv', encoding='utf-8-sig')

In [ ]:
import pandas as pd

df1=pd.read_csv('./104_comapny/jobresearch-數據分析.csv', encoding='utf-8-sig')
df2=df1.copy()

In [ ]:
#整理同義字
df2['SQL']=df2.loc[:,(df2.columns.str.contains('SQL'))].sum(axis=1)
df2['Linux']=df2.loc[:,(df2.columns.str.contains('Linux'or'ssh'or'Vmware'))].sum(axis=1)
df2['Excel/VBA']=df2.loc[:,(df2.columns.str.contains('Excel'or'VBA'))].sum(axis=1)
df2['C語言']=df2.loc[:,(df2.columns.str.contains('C' or 'C#' or 'C++'or 'C++.Net'))].sum(axis=1)
df2['JavaScript']=df2.loc[:,(df2.columns.str.contains('jQuery' or 'JSP' or 'JavaScript'or'Node.js' or 'D3.js'))].sum(axis=1)
df2['Hadoop']=df2.loc[:,(df2.columns.str.contains('Hadoop' or 'Hive'))].sum(axis=1)
df2['Oracle']=df2.loc[:,(df2.columns.str.contains('Oracle ERP' or 'Oracle Forms' or'Oracle'))].sum(axis=1)
df2['HTML/XML']=df2.loc[:,(df2.columns.str.contains('XHTML' or 'XML' or'XML Web services' or 'HTML'))].sum(axis=1)
df2['Git/Github']=df2.loc[:,(df2.columns.str.contains('Git' or 'Github'))].sum(axis=1)

In [ ]:
#刪除column→同義字欄位僅保留一個、刪除基本技能要求例如Office與打字能力
df2.drop(columns=['ssh', 'Vmware','VBA','C','C#','C++','C++.Net','jQuery','JSP','JavaScript','Node.js', \
                  'D3.js','Hive','hadoop','Oracle ERP','Oracle Forms','ANSI SQL', 'MS SQL', 'MySQL', 'PL/SQL', 'PostgreSQL',\
                  'Excel','中文打字20~50', '中文打字50~75', '英文打字20~50', '英文打字50~75', '英文打字75~100',\
                 'Windows 10', 'Windows 2000', 'Windows 7', 'Windows 8','Windows Server 2008', 'Windows Server 2012',\
                  'Windows XP','Word','PowerPoint','Outlook','Adobe Acrobat', 'Adobe InDesign', 'Adobe Photoshop', \
                  'Adobe XD','鼎新','Illustrator'], axis = 1, inplace=True)

In [ ]:
#將技能進行排序，依據出現次數作為權重評斷標準
df3=df2.iloc[:,8:]
df3.loc['Total'] = df3.sum(numeric_only=True)
df3 = df3.sort_values(by ='Total', axis=1,ascending=False)
df3 = df3.loc[:,(df3 >= 10).any()]
df3=df3.drop('Total')

df4=df2.iloc[:,1:8]
df5=pd.concat([df4,df3],axis=1)

In [ ]:
df6=df5.copy()
df6=df6.drop('Job_Description', axis=1)
df6=df6.drop('Other_Requirement', axis=1)

#依據「公司名稱」進行排序，將同一家公司開出的職缺放在一起，方便閱讀
df6 = df6.sort_values(by =['Company_Name'], axis=0)
df6.to_csv('./104_comapny/jobresearch_etl2.csv', encoding='utf-8-sig',index=False)
df6

,Company_Name,Job_Opening,Industry,Salary,Job_Url,Excel/VBA,SQL,Python,Google Analytics,R,C語言,Tableau,SAS,Java,SPSS,Linux,Power BI​,HTML,CSS,Google Tag Manager
0,(總公司)南山人壽保險股份有限公司,人力資源數據分析師/資深分析師,人身保險業,待遇面議,http://www.104.com.tw/job/3ejbo?jobsource=joli...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,(總公司)南山人壽保險股份有限公司,銀行通路業務訓練人員(TO),人身保險業,待遇面議,http://www.104.com.tw/job/4imiu?jobsource=joli...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17LIVE_英屬維京群島商藝啟股份有限公司臺灣分公司,(Jr./Sr.) Data Analyst 資深資料分析師,網際網路相關業,待遇面議,http://www.104.com.tw/job/736bz?jobsource=joli...,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17LIVE_英屬維京群島商藝啟股份有限公司臺灣分公司,Business Efficiency Analyst 商業分析師,網際網路相關業,待遇面議,http://www.104.com.tw/job/73vok?jobsource=joli...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17LIVE_英屬維京群島商藝啟股份有限公司臺灣分公司,Product Data Analyst 產品數據分析師,網際網路相關業,待遇面議,http://www.104.com.tw/job/701ac?jobsource=joli...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,魔髮部屋_沃生國際企業有限公司,SEO關鍵字廣告企劃,美髮業,"月薪40,000~50,000元",http://www.104.com.tw/job/71vlc?jobsource=joli...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
449,鴻海精密工業股份有限公司,數據分析工程師(內湖),消費性電子產品製造業,待遇面議,http://www.104.com.tw/job/5w7hx?jobsource=joli...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
450,麗嬰國際股份有限公司,開發部企劃人員,文具禮品相關業,"月薪30,000~32,000元",http://www.104.com.tw/job/70jsh?jobsource=joli...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
451,麟數據科技股份有限公司,社群數據分析師,電腦軟體服務業,"月薪30,000~42,000元",http://www.104.com.tw/job/4zlf9?jobsource=joli...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
